In [22]:
%matplotlib widget
import os,sys
import csv
import copy
import math
import collections

import matplotlib
import matplotlib.pyplot as plt

from clawpack import pyclaw
from clawpack import riemann
import numpy as np
from scipy.integrate import simps, trapz
from scipy.interpolate import interp1d

sys.path.append("/pymorph")
from schemes.weno import get_left_flux,get_right_flux
from schemes.weno import get_stencil
import sediment_transport.sed_trans as sedtrans

from models.shallow_water_solver import shallow_solver
from schemes.avalanche_scheme import *

from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd

from IPython.display import display, Math, Latex

%load_ext autoreload
%autoreload 2


plt.style.use(['dark_background'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
profile_folder = '/data/PhD/A3DOWN 2006-06-09 (S-800, 30-15cm)'

In [27]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    


def readWiebeFile(filepath, resolution = 0.5):
    xs = []
    zs = []
    with open(filepath) as f:
        for line in f:
            values = line.split(',')
            if is_number(values[0]) and is_number(values[1]):
                x = float(values[0])/100.
                z = float(values[1])/100.
                xs.append(x)
                zs.append(z)
                
    xs = np.array(xs)
    zs = np.array(zs)
    
    #--------------------------------
    # Increase the resolution on the grid
    #--------------------------------
    xmax = 12.0 #xs.max()
    nx = len(xs)
    f = interp1d(xs, zs)
    
    xnew = np.linspace(0., xmax, num=800)
    znew = f(xnew)
    return xnew, znew



    

In [28]:
xprofile = None
zdict = {}

for filename in os.listdir(profile_folder):
    name = filename.split('.')[0]
    profile = None
    if len(name)==10:
        profile = '0{0}'.format(name[-1:])
    else:
        profile = name[-2:]
    
    x, z = readWiebeFile(os.path.join(profile_folder,filename), resolution=0.25)
    
    xprofile = x
    zdict[profile] = z

profileDict = pd.DataFrame(zdict)

profileDict = profileDict.reindex(sorted(profileDict.columns), axis=1)

The x and y coordinates are in cms. 

In [29]:
profileDict.describe()

,00,01,02,03,04,05,06,07,08,09,...,25,26,27,28,29,30,31,32,33,34
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,...,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,-0.012777,-0.012027,-0.013002,-0.013411,-0.014287,-0.014384,-0.014101,-0.013310,-0.013846,-0.013487,...,-0.017836,-0.017153,-0.018468,-0.018861,-0.020548,-0.019215,-0.018510,-0.017467,-0.017857,-0.018646
std,0.033711,0.032189,0.032324,0.031508,0.029453,0.027400,0.026305,0.024873,0.024767,0.025190,...,0.020847,0.019999,0.018784,0.018903,0.018749,0.016816,0.015426,0.015127,0.015036,0.014664
min,-0.101800,-0.085600,-0.081575,-0.079800,-0.076122,-0.072161,-0.081734,-0.076785,-0.071947,-0.072874,...,-0.080835,-0.077800,-0.073833,-0.075128,-0.076924,-0.064500,-0.053853,-0.066407,-0.077600,-0.070711
25%,-0.037735,-0.035316,-0.035370,-0.035241,-0.035281,-0.033406,-0.030566,-0.029580,-0.031072,-0.027637,...,-0.027938,-0.024306,-0.025601,-0.025132,-0.031867,-0.030014,-0.029238,-0.025997,-0.025600,-0.028082
50%,-0.011210,-0.009602,-0.010907,-0.011178,-0.012298,-0.011176,-0.010255,-0.007697,-0.008162,-0.005914,...,-0.015544,-0.014757,-0.016093,-0.015795,-0.017981,-0.017001,-0.016134,-0.015090,-0.016373,-0.017792
75%,0.012210,0.010591,0.010667,0.010270,0.008596,0.008799,0.007543,0.007086,0.005900,0.006438,...,-0.000899,-0.001397,-0.004729,-0.004805,-0.005415,-0.006909,-0.007570,-0.006648,-0.006118,-0.007507
max,0.054977,0.055503,0.049971,0.045400,0.041310,0.035315,0.035495,0.032939,0.030519,0.027754,...,0.016783,0.015581,0.011970,0.010697,0.011996,0.012200,0.018400,0.010319,0.012200,0.007547


In [30]:
verts=[]
for column in profileDict:
    zc = profileDict[column].tolist()
    verts.append(list(zip(xprofile.copy(),zc.copy())))



In [31]:

lines = LineCollection(verts)
fig = plt.figure(figsize=(10, 10))
ax = fig.gca(projection='3d')
lines.set_alpha(0.7)
tsteps = np.linspace(0,34,35)
ax.add_collection3d(lines, zs=tsteps, zdir='y')
ax.set_xlim3d(0, 12.6)
ax.set_zlim3d(-0.250, .2)
ax.set_ylim3d(0, 34)
ax.set_zlabel('$z_b$')
ax.set_ylabel('$t(sec)$')
ax.set_xlabel('$x(m)$')


FigureCanvasNbAgg()

Text(0.5, 0, '$x(m)$')

In [32]:
lines.get_array

<bound method ScalarMappable.get_array of <mpl_toolkits.mplot3d.art3d.Line3DCollection object at 0x7f7332fd0710>>

In [33]:
plt.figure(1)
fig = plt.figure(figsize=(12, 2))
plt.plot(xprofile, profileDict['00'], 'bo')
plt.xlabel('station [m]')
plt.ylabel('elevation [m]')

FigureCanvasNbAgg()

Text(0, 0.5, 'elevation [m]')